In [0]:
%run ./EDP_Config

In [0]:
# dbutils.fs.rm("dbfs:/user/hive/warehouse/edp_dev.db/src_emp_brnz", True)
# qry = f"DROP TABLE edp_dev.src_emp_brnz"
# spark.sql(qry)

In [0]:
def load_table(df_cfg):
    try:
        tbl_nm = df_cfg['entity_nm']
        destination_table = f'edp_dev.{tbl_nm}_brnz'
        wtrmrkclm = df_cfg['wtr_mrk_clm_nm']
        wtrmrkqry = df_cfg['src_wtr_mrk_qry']
        lowerbound = df_cfg['wtr_mrk_clm_val']
        wtrmrkupqry = df_cfg['cfg_upd_wtr_mrk_qry']
        src_qry = df_cfg['clm_dtls']
        LdSrtTimenrws = datetime.now().isoformat()

        # Source watermark
        wtrmrkqry = wtrmrkqry.format(wtrmrkclm, tbl_nm).replace('"', "")
        wtrmrk = spark.sql(wtrmrkqry).collect()[0][0]
        print(tbl_nm)

        # Prepare the Source Query for Incremental Load
        src_qry = src_qry.format(lowerbound, wtrmrk)
        print(src_qry)

        # Dataframe to execute the SQL query from configuration table
        df = spark.sql(src_qry)
        print(df.count())

        if df.isEmpty():
            # Log message in the log table if no rows returned
            MsgDescnorws = f'No incremental rows exists for the table : {tbl_nm}  , with watermark : {lowerbound} in the source Infor LN'
            LdEndTimenrws = datetime.now().isoformat()
            log_qry_no_rws = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)
                                 VALUES (null, '{tbl_nm}', null, null, 0, 'Failure', 0, null, null, null, '{LdSrtTimenrws}', '{LdEndTimenrws}', null, 'brnz', '{wtrmrk}', '{LdEndTimenrws}')"""
            spark.sql(log_qry_no_rws)
        else:
            # Define the execution log variable
            DataRead = df.count()
            RowsCopied = DataRead
            ExecutionStatus = 'Success'
            ZoneInfo = 'bronze'
            LdEndTimenrws = datetime.now().isoformat()

            # Write the source data in the table with Schema Overwrite
            df.write.format("delta").mode("append").option("overwriteSchema", "True").saveAsTable(destination_table)
            MsgDescription = f'Data loaded successfully for the table : {tbl_nm}  . Last Watermark value fetched from source successfully for the table : {tbl_nm}  . with value: {wtrmrk} '

            # Update watermark in the metadata table
            wtrmrkupqry = wtrmrkupqry.replace('"', "'").replace('"', "'")
            updateqry = wtrmrkupqry.format(wtrmrk, tbl_nm)
            spark.sql(updateqry)
            MsgDescription += f'Watermark updated successfully in the metadata table : {tbl_nm}  . with value: {wtrmrk} '

            # Log message in the log table
            log_qry_success = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)
                                  VALUES (null, '{tbl_nm}', null, null, {DataRead}, '{ExecutionStatus}', {RowsCopied}, null, null, null, '{LdSrtTimenrws}', '{LdEndTimenrws}', null, 'brnz', '{wtrmrk}', '{LdEndTimenrws}')"""
            spark.sql(log_qry_success)
            
    except Exception as e:
        print(e)
        tbl_nm=df_cfg['entity_nm']
        LdSrtTimenrws=datetime.now().isoformat()
        LdEndTimenrws=datetime.now().isoformat()
        ## Source watermark
        wtrmrkqry=f"{wtrmrkqry}".format(wtrmrkclm,tbl_nm)
        wtrmrkqry=wtrmrkqry.replace('"', "")
        wtrmrkqry=f'{wtrmrkqry}'
        wtrmrk=spark.sql(wtrmrkqry).collect()[0][0]

        DataRead = 0
        RowsCopied=DataRead

        tabl_nm = df_cfg['entity_nm']
        exec_status = "Failed"
        zn_info = "bronze"
        err_cd = None
        err_desc = f"{e}"
        err_desc = err_desc.replace("'", "\\'")
        err_lg_dt_time = tstampiso
        
        log_qry_failure = f"""INSERT INTO EDP_DEV.process_inventory_log (src_nm, entity_nm, pip_nm, run_id, src_rec_cnt, dl_cpy_status,dl_rec_cnt, dl_fs_nm, dl_dr_nm,dl_fl_nm, ing_strt_time, ing_end_time, pip_type, zn_info, wtr_mrk_cur_val, ld_dt_time)VALUES (null, '{tbl_nm}', null,null, {DataRead},'{exec_status}', {RowsCopied},null,null,null,'{LdSrtTimenrws}','{LdEndTimenrws}',null,'brnz','{wtrmrk}', '{LdEndTimenrws}')"""
        spark.sql(log_qry_failure)

        log_qry_err = f"""INSERT INTO EDP_DEV.error_log (src_nm, entity_nm, uc_nm, sch_nm, exec_status, zn_info, err_cd, err_desc, err_lg_dt_time, obj_type) VALUES (null, '{tbl_nm}', null, null, '{exec_status}', '{zn_info}', '{err_cd}', '{err_desc}', '{err_lg_dt_time}', null)"""
        spark.sql(log_qry_err)



In [0]:
qry=f"""select * from edp_dev.incremental_load_cfg """
df_cfg=spark.sql(qry)
df_cfg.select(col('src_nm'),col('entity_nm'),col('wtr_mrk_clm_nm'),col('wtr_mrk_clm_val'),col('part_clm_nm'),col('part_clm_val'),col('interval'),col('frequency'), col('clm_dtls'),col('ref_qry'),col('src_wtr_mrk_qry'),col('cfg_upd_wtr_mrk_qry'),col('batch_sz'),col('ld_seq'),col('uc_nm'),col('sch_nm'),col('mig_flag'))

dic = df_cfg.collect()
dic = [row.asDict() for row in dic]
print(dic)

[{'src_nm': '', 'entity_nm': 'src_emp', 'wtr_mrk_clm_nm': 'modifieddt', 'wtr_mrk_clm_val': '1900-01-01T13:32:31.893048', 'src_clm_key': 'Emp_ID', 'part_clm_nm': '', 'part_clm_val': '', 'interval': 1, 'frequency': 1, 'clm_dtls': '(Select Emp_Id, Emp_nm, Dept_Id, createdt, modifieddt, modifieddt as watermark from EDP_source.src_emp where modifieddt > "{}"  and modifieddt <="{}")', 'ref_qry': '', 'src_wtr_mrk_qry': 'select max("{}") from EDP_source."{}"', 'cfg_upd_wtr_mrk_qry': 'update EDP_DEV.incremental_load_cfg set wtr_mrk_clm_val="{}" where entity_nm="{}"', 'batch_sz': '', 'ld_seq': None, 'uc_nm': '', 'sch_nm': '', 'mig_flag': ''}]


In [0]:
from threading import Thread
from queue import Queue

q = Queue()

worker_count = worker_count
def run_tasks(function, q):
    while not q.empty():
        try:
            value = q.get()
            function(value)
        except Exception as e:
            output=f'{e}'
            print(output)
    
        finally:
         q.task_done()

for table in  dic:
   q.put(table)

In [0]:
for i in range(driver_count):
    try:
        t=Thread(target=run_tasks, args=(load_table, q))
        t.daemon = True
        t.start()
    except Exception as e: 
        output=f'{e}'
        print(output)

q.join()

src_emp
(Select Emp_Id, Emp_nm, Dept_Id, createdt, modifieddt, modifieddt as watermark from EDP_source.src_emp where modifieddt > "1900-01-01T13:32:31.893048"  and modifieddt <="2024-07-02T06:06:48.959820")
23
